# Random Forest Models: Classification and Regression

This is the R notebook for the Random Forest based models for the trip analysis. 

The following are the required libraries to run the models:

In [1]:
library(randomForest)
library(mlbench)
library(caret)
library(e1071)
library(dplyr)
library(tidyr)
library(readr)
library(ranger)
library(rFerns)
library(ordinalForest)
library(RRF)
library(future)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘dplyr’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance


Registered S3 method overwritten by 'RRF':
  method      from        
  plot.margin randomForest

RRF 1.9.1

Type rrfNews() to see new features/changes/bug fixes.


Attaching package: ‘RRF’


The following object is masked from ‘package:ranger’:

    importance


## Data Input

In [2]:
carta = read_csv('data/jmartinez/Transit_Data/carta.csv', progress = F) 


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  scheduled_arrival_time = col_datetime(format = ""),
  actual_arrival_time = col_datetime(format = ""),
  direction_desc = col_character(),
  service_period = col_character(),
  date = col_date(format = ""),
  scheduled_datetime = col_datetime(format = ""),
  actual_arrival_datetime = col_datetime(format = ""),
  trip_start_time = col_datetime(format = ""),
  trip_date = col_date(format = ""),
  service_kind = col_character()
)
ℹ Use `spec()` for the full column specifications.


Warning message:
“1658648 parsing failures.
    row      col               expected actual                                    file
5820871 route_id no trailing characters    10A 'data/jmartinez/Transit_Data/carta.csv'
5820872 route_id no trailing characters    10A 'data/jmartinez/Transit_Data/carta.csv'
5820873 route_id no trailing characters    10A 'data/jmartinez/Transit_Data/carta.csv'
5820874

In [4]:
carta$trip_id = as.character(carta$trip_id)
carta$stop_id = as.character(carta$stop_id)
carta$route_id = as.character(carta$route_id)
carta$direction_id = as.character(carta$direction_id)
carta$hour = factor(carta$hour)
carta$month = factor(carta$month)

In [5]:
head(carta)

trip_id,scheduled_arrival_time,actual_arrival_time,stop_id,stop_sequence,stop_lat,stop_lon,route_id,direction_id,board_count,⋯,actual_arrival_datetime,trip_start_time,day_of_week,trip_date,hour,year,month,Estimated_Temp,Estimated_Precip,service_kind
<chr>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,⋯,<dttm>,<dttm>,<dbl>,<date>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<chr>
132994,1900-01-01 04:55:00,1900-01-01 04:52:00,1351,1,35.05266,-85.30972,1,0,1,⋯,2019-01-02 04:52:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.350000,0,weekday
132994,1900-01-01 04:55:41,1900-01-01 04:56:00,12,2,35.05070,-85.30953,1,0,0,⋯,2019-01-02 04:56:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.343167,0,weekday
132994,1900-01-01 04:56:20,1900-01-01 04:57:00,1555,3,35.04883,-85.30955,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.336667,0,weekday
132994,1900-01-01 04:56:35,1900-01-01 04:57:00,1579,4,35.04812,-85.30955,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.334167,0,weekday
132994,1900-01-01 04:57:22,1900-01-01 04:57:00,1354,5,35.04598,-85.30952,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.326333,0,weekday
132994,1900-01-01 04:57:35,1900-01-01 04:57:00,1353,6,35.04538,-85.30945,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.324167,0,weekday


We create the data from the main data set as follows:

# Route 1

## Direction 0

### Pre-lockdown

In [6]:
Pre_Trips_r1_d0 <- carta %>%
  filter(trip_date < '2020-03-05', direction_id == 0, route_id == '1') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Pre_Trips_r1_d0 <- unique(Pre_Trips_r1_d0)

Pre_Trips_r1_d0$trip_id = factor(Pre_Trips_r1_d0$trip_id)
Pre_Trips_r1_d0$hour = factor(Pre_Trips_r1_d0$hour)
Pre_Trips_r1_d0$mean_temp = (Pre_Trips_r1_d0$mean_temp - min(Pre_Trips_r1_d0$mean_temp))/(max(Pre_Trips_r1_d0$mean_temp) - min(Pre_Trips_r1_d0$mean_temp))
Pre_Trips_r1_d0$mean_precip = (Pre_Trips_r1_d0$mean_precip - min(Pre_Trips_r1_d0$mean_precip))/(max(Pre_Trips_r1_d0$mean_precip) - min(Pre_Trips_r1_d0$mean_precip))

head(Pre_Trips_r1_d0)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2019-01-02,132994,4,1,0.3288005,0
2019-01-02,132994,5,2,0.3283889,0
2019-01-02,132997,11,7,0.3535107,0
2019-01-02,132998,5,3,0.3283889,0
2019-01-02,133002,6,3,0.2918481,0
2019-01-02,133002,7,3,0.2940220,0


### Post-lockdown

In [8]:
Pre_Trips_r1_d1 <- carta %>%
  filter(trip_date < '2020-03-05', route_id == '1', direction_id == '1') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Pre_Trips_r1_d1 <- unique(Pre_Trips_r1_d1)

Pre_Trips_r1_d1$trip_id = factor(Pre_Trips_r1_d1$trip_id)
Pre_Trips_r1_d1$hour = factor(Pre_Trips_r1_d1$hour)

Pre_Trips_r1_d1$mean_temp = (Pre_Trips_r1_d1$mean_temp - min(Pre_Trips_r1_d1$mean_temp))/(max(Pre_Trips_r1_d1$mean_temp) - min(Pre_Trips_r1_d1$mean_temp))
Pre_Trips_r1_d1$mean_precip = (Pre_Trips_r1_d1$mean_precip - min(Pre_Trips_r1_d1$mean_precip))/(max(Pre_Trips_r1_d1$mean_precip) - min(Pre_Trips_r1_d1$mean_precip))

head(Pre_Trips_r1_d1)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2019-01-02,133039,5,17,0.3279331,0
2019-01-02,133041,6,14,0.3138050,0
2019-01-02,133045,7,11,0.3088654,0
2019-01-02,133045,8,10,0.3164356,0
2019-01-02,133046,8,4,0.3209698,0
2019-01-02,133048,9,6,0.3502474,0


## Post-lockdown

### Direction 0

In [9]:
Post_Trips_r1_d0 <- carta %>%
  filter(trip_date >= '2020-03-05', route_id == '1', direction_id == '0') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Post_Trips_r1_d0 <- unique(Post_Trips_r1_d0)

Post_Trips_r1_d0$trip_id = factor(Post_Trips_r1_d0$trip_id)
Post_Trips_r1_d0$hour = factor(Post_Trips_r1_d0$hour)

Post_Trips_r1_d0$mean_temp = (Post_Trips_r1_d0$mean_temp - min(Post_Trips_r1_d0$mean_temp))/(max(Post_Trips_r1_d0$mean_temp) - min(Post_Trips_r1_d0$mean_temp))
Post_Trips_r1_d0$mean_precip = (Post_Trips_r1_d0$mean_precip - min(Post_Trips_r1_d0$mean_precip))/(max(Post_Trips_r1_d0$mean_precip) - min(Post_Trips_r1_d0$mean_precip))

head(Post_Trips_r1_d0)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2020-03-05,138338,5,0,0.2916571,0
2020-03-05,138342,5,3,0.2916571,0
2020-03-05,138342,6,6,0.2898337,0
2020-03-05,138344,6,1,0.2761706,0
2020-03-05,138346,7,10,0.2796350,0
2020-03-05,138347,7,2,0.2857127,0


## Direction 1

In [11]:
Post_Trips_r1_d1 <- carta %>%
  filter(trip_date >= '2020-03-05', route_id == '1', direction_id == '1') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Post_Trips_r1_d1 <- unique(Post_Trips_r1_d1)

Post_Trips_r1_d1$trip_id = factor(Post_Trips_r1_d1$trip_id)
Post_Trips_r1_d1$hour = factor(Post_Trips_r1_d1$hour)

Post_Trips_r1_d1$mean_temp = (Post_Trips_r1_d1$mean_temp - min(Post_Trips_r1_d1$mean_temp))/(max(Post_Trips_r1_d1$mean_temp) - min(Post_Trips_r1_d1$mean_temp))
Post_Trips_r1_d1$mean_precip = (Post_Trips_r1_d1$mean_precip - min(Post_Trips_r1_d1$mean_precip))/(max(Post_Trips_r1_d1$mean_precip) - min(Post_Trips_r1_d1$mean_precip))

head(Post_Trips_r1_d1)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2020-03-05,138383,5,4,0.2906203,0
2020-03-05,138383,6,4,0.2873011,0
2020-03-05,138385,6,5,0.2757920,0
2020-03-05,138385,7,5,0.2711697,0
2020-03-05,138387,7,5,0.2782163,0
2020-03-05,138389,8,6,0.2989043,0


# Route 4

## Direction 0

### Pre-lockdown 

In [21]:
Pre_Trips_r4_d0 <- carta %>%
  filter(trip_date < '2020-03-05', route_id == '4', direction_id == '0') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Pre_Trips_r4_d0 <- unique(Pre_Trips_r4_d0)

Pre_Trips_r4_d0$trip_id = factor(Pre_Trips_r4_d0$trip_id)
Pre_Trips_r4_d0$hour = factor(Pre_Trips_r4_d0$hour)

Pre_Trips_r4_d0$mean_temp = (Pre_Trips_r4_d0$mean_temp - min(Pre_Trips_r4_d0$mean_temp))/(max(Pre_Trips_r4_d0$mean_temp) - min(Pre_Trips_r4_d0$mean_temp))
Pre_Trips_r4_d0$mean_precip = (Pre_Trips_r4_d0$mean_precip - min(Pre_Trips_r4_d0$mean_precip))/(max(Pre_Trips_r4_d0$mean_precip) - min(Pre_Trips_r4_d0$mean_precip))

head(Pre_Trips_r4_d0)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2019-01-02,134296,4,0,0.3292844,0
2019-01-02,134296,5,5,0.3287357,0
2019-01-02,134298,8,16,0.3243253,0
2019-01-02,134298,9,9,0.3316438,0
2019-01-02,134299,9,21,0.3423543,0
2019-01-02,134299,10,9,0.3538335,0


## Direction 1

In [22]:
Pre_Trips_r4_d1 <- carta %>%
  filter(trip_date < '2020-03-05', route_id == '4', direction_id == '1') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Pre_Trips_r4_d1 <- unique(Pre_Trips_r4_d1)

Pre_Trips_r4_d1$trip_id = factor(Pre_Trips_r4_d1$trip_id)
Pre_Trips_r4_d1$hour = factor(Pre_Trips_r4_d1$hour)

Pre_Trips_r4_d1$mean_temp = (Pre_Trips_r4_d1$mean_temp - min(Pre_Trips_r4_d1$mean_temp))/(max(Pre_Trips_r4_d1$mean_temp) - min(Pre_Trips_r4_d1$mean_temp))
Pre_Trips_r4_d1$mean_precip = (Pre_Trips_r4_d1$mean_precip - min(Pre_Trips_r4_d1$mean_precip))/(max(Pre_Trips_r4_d1$mean_precip) - min(Pre_Trips_r4_d1$mean_precip))

head(Pre_Trips_r4_d1)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2019-01-02,134364,6,11,0.3073824,0
2019-01-02,134365,9,17,0.3450123,0
2019-01-02,134365,10,17,0.3525516,0
2019-01-02,134366,9,8,0.3457845,0
2019-01-02,134366,10,13,0.3524518,0
2019-01-02,134368,7,24,0.3077242,0


### Post-lockdown

In [23]:
Post_Trips_r4_d0 <- carta %>%
  filter(trip_date >= '2020-03-05', route_id == '4', direction_id == '0') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Post_Trips_r4_d0 <- unique(Post_Trips_r4_d0)

Post_Trips_r4_d0$trip_id = factor(Post_Trips_r4_d0$trip_id)
Post_Trips_r4_d0$hour = factor(Post_Trips_r4_d0$hour)

Post_Trips_r4_d0$mean_temp = (Post_Trips_r4_d0$mean_temp - min(Post_Trips_r4_d0$mean_temp))/(max(Post_Trips_r4_d0$mean_temp) - min(Post_Trips_r4_d0$mean_temp))
Post_Trips_r4_d0$mean_precip = (Post_Trips_r4_d0$mean_precip - min(Post_Trips_r4_d0$mean_precip))/(max(Post_Trips_r4_d0$mean_precip) - min(Post_Trips_r4_d0$mean_precip))

head(Post_Trips_r4_d0)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2020-03-05,139636,4,0,0.3121739,0
2020-03-05,139636,5,3,0.3121739,0
2020-03-05,139642,9,14,0.3476467,0
2020-03-05,139647,7,13,0.3005615,0
2020-03-05,139654,5,0,0.3121739,0
2020-03-05,139654,6,1,0.3067751,0


## Direction 1

In [24]:
Post_Trips_r4_d1 <- carta %>%
  filter(trip_date >= '2020-03-05', route_id == '4', direction_id == '1') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Post_Trips_r4_d1 <- unique(Post_Trips_r4_d1)

Post_Trips_r4_d1$trip_id = factor(Post_Trips_r4_d1$trip_id)
Post_Trips_r4_d1$hour = factor(Post_Trips_r4_d1$hour)

Post_Trips_r4_d1$mean_temp = (Post_Trips_r4_d1$mean_temp - min(Post_Trips_r4_d1$mean_temp))/(max(Post_Trips_r4_d1$mean_temp) - min(Post_Trips_r4_d1$mean_temp))
Post_Trips_r4_d1$mean_precip = (Post_Trips_r4_d1$mean_precip - min(Post_Trips_r4_d1$mean_precip))/(max(Post_Trips_r4_d1$mean_precip) - min(Post_Trips_r4_d1$mean_precip))

head(Post_Trips_r4_d1)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2020-03-05,139698,6,24,0.2631616,0
2020-03-05,139698,7,5,0.2535160,0
2020-03-05,139704,9,8,0.3345024,0
2020-03-05,139704,10,11,0.3345024,0
2020-03-05,139708,6,0,0.2556453,0
2020-03-05,139708,7,10,0.2637675,0


# Route 9

## Direction 0

### Pre-lockdown

In [30]:
Pre_Trips_r9_d0 <- carta %>%
  filter(trip_date < '2020-03-05', route_id == '9', direction_id == '0') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Pre_Trips_r9_d0 = unique(Pre_Trips_r9_d0)

Pre_Trips_r9_d0$trip_id = factor(Pre_Trips_r9_d0$trip_id)
Pre_Trips_r9_d0$hour = factor(Pre_Trips_r9_d0$hour)

Pre_Trips_r9_d0$mean_temp = (Pre_Trips_r9_d0$mean_temp - min(Pre_Trips_r9_d0$mean_temp))/(max(Pre_Trips_r9_d0$mean_temp) - min(Pre_Trips_r9_d0$mean_temp))
Pre_Trips_r9_d0$mean_precip = (Pre_Trips_r9_d0$mean_precip - min(Pre_Trips_r9_d0$mean_precip))/(max(Pre_Trips_r9_d0$mean_precip) - min(Pre_Trips_r9_d0$mean_precip))

head(Pre_Trips_r9_d0)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2019-01-02,134767,4,0,0.3286791,0
2019-01-02,134767,5,0,0.3281117,0
2019-01-02,134768,5,1,0.3281117,0
2019-01-02,134769,7,7,0.3110232,0
2019-01-02,134769,8,4,0.3170004,0
2019-01-02,134770,8,3,0.3272536,0


## Direction 1

In [36]:
Pre_Trips_r9_d1 <- carta %>%
  filter(trip_date < '2020-03-05', route_id == '9', direction_id == '1') %>%
  group_by(trip_date, trip_id, hour) %>%
  mutate(max_occupancy = max(occupancy),
         mean_temp = mean(Estimated_Temp),
         mean_precip = mean(Estimated_Precip),
         max_occupancy = if_else(max_occupancy < 0, 0, max_occupancy),
         max_occupancy = if_else(max_occupancy > 32, 32, max_occupancy)) %>%
  summarise(max_occupancy, mean_temp, mean_precip)

Pre_Trips_r9_d1 <- unique(Pre_Trips_r9_d1)

Pre_Trips_r9_d1$trip_id = factor(Pre_Trips_r9_d1$trip_id)
Pre_Trips_r9_d1$hour = factor(Pre_Trips_r9_d1$hour)

Pre_Trips_r9_d1$mean_temp = (Pre_Trips_r9_d1$mean_temp - min(Pre_Trips_r9_d1$mean_temp))/(max(Pre_Trips_r9_d1$mean_temp) - min(Pre_Trips_r9_d1$mean_temp))
Pre_Trips_r9_d1$mean_precip = (Pre_Trips_r9_d1$mean_precip - min(Pre_Trips_r9_d1$mean_precip))/(max(Pre_Trips_r9_d1$mean_precip) - min(Pre_Trips_r9_d1$mean_precip))

head(Pre_Trips_r9_d1)

`summarise()` has grouped output by 'trip_date', 'trip_id', 'hour'. You can override using the `.groups` argument.



trip_date,trip_id,hour,max_occupancy,mean_temp,mean_precip
<date>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
2019-01-02,134796,5,12,0.3284620,0
2019-01-02,134796,6,5,0.3284620,0
2019-01-02,134797,6,10,0.3208408,0
2019-01-02,134798,10,6,0.3535623,0
2019-01-02,134799,11,7,0.3535623,0
2019-01-02,134799,12,5,0.3535623,0


# Vanilla Random Forest (Ranger)

The function `vanlla_rf_trips_cv()` takes the following inputs:

    * `rt`: route id.
    * `di`: direction id.
    * `part`: Pre- or post-lockdown.
    * `trip`: trip id.
    
And returns a Random Forest (Ranger) model for the pre-defined conditions.

In [16]:
vanilla_rf_trips_cv <- function(rt, di, part, trip){
    
    library(caret)
    library(ranger)
        
    setwd('/home/jmartinez')
    
    file_dir = paste('data', 'jmartinez', 'Max_Occupancy_Trips', 'data',
                     sep = '/')
    
    file_name = paste0(part, '_', 'Trips', '_', 'r', rt, '_', 'd', di, '.csv')
    
    file_path = paste(file_dir, file_name, sep = '/')
    
    DF = read_csv(file_path)
    
    trip_df <- DF %>%
        filter(trip_id == trip)
    trip_df <- unique(trip_df)
    
    num_rows_dt = nrow(trip_df)
    
    if(num_rows_dt == 0){
        return('Insufficient data for analysis!')
    }
    
    else{
         
        num_zeros = dim(trip_df %>% filter(max_occupancy == 0))[1]

        set.seed(123)
        inTraining <-  createDataPartition(trip_df$max_occupancy,
                                           times = 1, p = 0.7,
                                           list = FALSE)
        
        DT_train <- trip_df[inTraining, ]
        DT_test <- trip_df[-inTraining, ]

        num_hours = length(unique(trip_df$hour))

        if(num_hours < 2){

            dt_train = DT_train[ , -c(1:3)]
            dt_test = DT_test[ , -c(1:3)]
        }
        else{

            dt_train = DT_train[, -c(1:2)]
            dt_test = DT_test[, -c(1:2)]
        }

        setwd('data/jmartinez/Max_Occupancy_Trips/Results')
        results_path = paste(paste('route', rt, sep = '_'),
                             paste(part, 'lockdown', sep = '_'),
                             paste('direction', di, sep = ''),
                             sep = '/')

        setwd(results_path)

        #dir.create(trip)

        n_row_train = nrow(DT_train)

        if(n_row_train < 30){

            return('Insufficient data for analysis!')
        }
        else if(n_row_train >= 30){
            
                                           
            control <- trainControl(method = 'cv',
                                    number = 5, 
                                    search = 'random')
                
            set.seed(123)
            rf_reg_ranger <- train(max_occupancy ~ .,
                                   data = dt_train,
                                   method = 'ranger',
                                   metric = 'RMSE',
                                   tuneLength  = 20, 
                                   trControl = control)              
                
            #----------------------------------------------------------------------------------
            # Validation:

            rf_vanilla_pred <- predict(rf_reg_ranger, newdata = dt_test)
            dt_test$RF_Vanilla_Pred = rf_vanilla_pred
            DT_test$RF_Vanilla_Pred = rf_vanilla_pred

            RF_train_RMSE = rf_reg_ranger$results$RMSE[1]
            RF_test_RMSE = sqrt(mean((dt_test$max_occupancy - dt_test$RF_Vanilla_Pred)^{2}))

            Performance = data.frame('Train' = RF_train_RMSE, 'Test'= RF_test_RMSE)

            # Predicitons on all available data:

            RF_RMSEs = data.frame('Train' = RF_train_RMSE, 'Test' = RF_test_RMSE)

            rownames(RF_RMSEs) = c('Vanilla_RF')

            file_DT_train_name = paste(trip, paste(part, 'DT_train.csv', sep = '_'), sep = '/')
            file_DT_test_name = paste(trip, paste(part, 'DT_test.csv', sep = '_'), sep = '/')

            file_name_RMSEs = paste(trip, paste(part, 'RF_RMSEs.csv', sep = '_'), sep = '/')
            file_name_Predictions = paste(trip, paste(part, 'Chart_Predictions.csv', sep = '_'), sep = '/')

            final_rf_vanilla = paste(trip, paste(part, 'rf_vanilla.rds', sep = '_'), sep = '/')
                
            write.csv(RF_RMSEs, file_name_RMSEs)
            write.csv(DT_test, file_name_Predictions, row.names = F)
                
            write.csv(DT_train, file_DT_train_name, row.names = F)
            write.csv(DT_test, file_DT_test_name, row.names = F)

            saveRDS(rf_reg_ranger, final_rf_vanilla)

        }
           
        setwd('/home/jmartinez')
        msg = paste('Done!', trip, 'had', num_zeros, 'in total.')    
        return(msg)
    }
}

# Results

The following sections have the results of all the trips for each combination of route, direction id, and data partition:

# Route 1

## Direction 0 

### Pre-lockdown

In [14]:
Pre_trip_id_r1_d0 = as.character(unique(Pre_Trips_r1_d0$trip_id))
Pre_trip_id_r1_d0

[1] "132994" "132997" "132998" "133002" "133003" "133004" "133005" "133007"
  [9] "133008" "133009" "133010" "133011" "133013" "133014" "133015" "133018"
 [17] "133024" "133025" "133026" "133028" "133034" "133036" "133038" "132995"
 [25] "133006" "133012" "133016" "133017" "133019" "133020" "133021" "133027"
 [33] "133032" "133033" "133035" "133037" "133001" "133022" "133023" "133029"
 [41] "133031" "133086" "133087" "133089" "133093" "133101" "133104" "133107"
 [49] "133109" "133110" "133112" "133113" "133114" "133115" "133116" "133117"
 [57] "133154" "133156" "133158" "133160" "133162" "132996" "132999" "133000"
 [65] "133030" "133084" "133091" "133095" "133097" "133099" "133100" "133102"
 [73] "133118" "133155" "133157" "133159" "133161" "133163" "133085" "133090"
 [81] "133092" "133094" "133096" "133098" "133105" "133088" "133103" "133108"
 [89] "135825" "135826" "135827" "135828" "135830" "135831" "135832" "135834"
 [97] "135665" "135666" "135667" "135668" "135669" "135671" "135672" "135673"
[105] "135675" "135676" "135677" "135680" "135681" "135683" "135684" "135685"
[113] "135686" "135687" "135688" "135689" "135690" "135691" "135692" "135693"
[121] "135694" "135697" "135699" "135700" "135702" "135703" "135704" "135707"
[129] "135708" "135670" "135679" "135695" "135701" "135705" "135706" "135709"
[137] "135674" "135678" "135696" "135755" "135756" "135757" "135758" "135759"
[145] "135760" "135761" "135763" "135764" "135765" "135766" "135768" "135769"
[153] "135771" "135772" "135773" "135774" "135775" "135776" "135777" "135778"
[161] "135779" "135780" "135781" "135782" "135783" "135784" "135785" "135786"
[169] "135788" "135789" "135833" "135770" "135787" "135829" "135682" "138338"
[177] "138372" "138339" "138341" "138344" "138349" "138351" "138352" "138353"
[185] "138354" "138357" "138358" "138362" "138363" "138368" "138369" "138370"
[193] "138337" "138340" "138342" "138345" "138346" "138347" "138350" "138355"
[201] "138356" "138359" "138360" "138361" "138365" "138366" "138367" "138373"
[209] "138375" "138343" "138374" "138376" "138377" "138378" "138380" "138381"
[217] "138371" "138427" "138429" "138430" "138431" "138434" "138435" "138438"
[225] "138439" "138442" "138443" "138445" "138446" "138447" "138450" "138451"
[233] "138452" "138455" "138457" "138459" "138461" "138497" "138499" "138501"
[241] "138502" "138503" "138504" "138506" "138379" "138428" "138432" "138433"
[249] "138436" "138437" "138440" "138441" "138444" "138448" "138500" "138505"
[257] "138453" "138456" "138458" "138460" "138498"

In [ ]:
# 

In [ ]:
which(Pre_trip_id_r1_d0 == '', arr.ind = T)

In [17]:
for(i in Pre_trip_id_r1_d0[1:261]){
  print(i)
  vanilla_rf_trips_cv('1', '0', 'Pre', i)
  print('Done!')
  print('_*_*_')
}

[1] "132994"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "132997"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "132998"


...

[1] "Done!"
[1] "_*_*_"
[1] "138460"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "138498"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"


## Direction 1

In [18]:
Pre_trip_id_r1_d1 = as.character(unique(Pre_Trips_r1_d1$trip_id))
Pre_trip_id_r1_d1

[1] "133039" "133041" "133045" "133046" "133048" "133049" "133050" "133052"
  [9] "133053" "133054" "133055" "133056" "133058" "133059" "133060" "133063"
 [17] "133064" "133069" "133070" "133071" "133073" "133074" "133075" "133076"
 [25] "133078" "133079" "133081" "133083" "133040" "133042" "133047" "133051"
 [33] "133057" "133061" "133062" "133065" "133066" "133067" "133068" "133072"
 [41] "133077" "133080" "133082" "133044" "133123" "133126" "133127" "133129"
 [49] "133134" "133136" "133137" "133140" "133141" "133143" "133145" "133146"
 [57] "133148" "133149" "133150" "133151" "133152" "133153" "133164" "133166"
 [65] "133168" "133170" "133172" "133043" "133120" "133122" "133131" "133133"
 [73] "133135" "133138" "133165" "133167" "133169" "133171" "133173" "133119"
 [81] "133121" "133124" "133125" "133130" "133132" "133128" "133139" "133144"
 [89] "135835" "135836" "135837" "135838" "135839" "135840" "135841" "135842"
 [97] "135843" "135844" "135710" "135711" "135712" "135713" "135714" "135715"
[105] "135716" "135717" "135718" "135719" "135720" "135721" "135722" "135723"
[113] "135724" "135725" "135726" "135728" "135729" "135730" "135731" "135732"
[121] "135733" "135734" "135735" "135736" "135737" "135738" "135740" "135742"
[129] "135743" "135746" "135747" "135749" "135750" "135751" "135752" "135753"
[137] "135744" "135745" "135748" "135754" "135741" "135790" "135791" "135792"
[145] "135793" "135794" "135795" "135797" "135798" "135799" "135800" "135802"
[153] "135803" "135804" "135805" "135806" "135807" "135808" "135809" "135810"
[161] "135811" "135812" "135813" "135814" "135815" "135816" "135818" "135819"
[169] "135820" "135821" "135822" "135823" "135824" "135817" "135727" "138383"
[177] "138417" "138384" "138385" "138387" "138390" "138392" "138393" "138394"
[185] "138398" "138399" "138402" "138403" "138407" "138408" "138413" "138414"
[193] "138415" "138382" "138388" "138389" "138391" "138395" "138396" "138400"
[201] "138401" "138404" "138405" "138406" "138410" "138411" "138412" "138418"
[209] "138420" "138386" "138416" "138419" "138421" "138422" "138423" "138424"
[217] "138426" "138425" "138463" "138464" "138465" "138469" "138470" "138474"
[225] "138475" "138476" "138478" "138481" "138482" "138483" "138484" "138486"
[233] "138487" "138488" "138491" "138493" "138495" "138507" "138509" "138511"
[241] "138512" "138513" "138514" "138515" "138516" "138462" "138466" "138467"
[249] "138468" "138471" "138472" "138473" "138477" "138510" "138480" "138489"
[257] "138492" "138494" "138496" "138508"

In [ ]:
# 

In [25]:
length(Pre_trip_id_r1_d1)

[1] 260

In [20]:
for(i in Pre_trip_id_r1_d1[1:260]){
  print(i)
  vanilla_rf_trips_cv('1', '1', 'Pre', i)
  print('Done!')
  print('_*_*_')
}

[1] "133039"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "133041"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "133045"


...

[1] "Done!"
[1] "_*_*_"
[1] "138496"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "138508"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"


In [26]:
Pre_trip_id_r4_d0 = as.character(unique(Pre_Trips_r4_d0$trip_id))
Pre_trip_id_r4_d0

[1] "134296" "134298" "134299" "134300" "134302" "134303" "134304" "134305"
  [9] "134306" "134307" "134309" "134310" "134311" "134312" "134313" "134314"
 [17] "134315" "134316" "134317" "134318" "134319" "134322" "134323" "134326"
 [25] "134327" "134328" "134329" "134330" "134332" "134333" "134335" "134336"
 [33] "134337" "134338" "134341" "134344" "134345" "134346" "134347" "134348"
 [41] "134349" "134350" "134351" "134354" "134355" "134356" "134357" "134360"
 [49] "134361" "134362" "134301" "134320" "134324" "134325" "134334" "134339"
 [57] "134340" "134342" "134343" "134352" "134353" "134358" "134359" "134363"
 [65] "134297" "134308" "134321" "134331" "134432" "134434" "134436" "134438"
 [73] "134440" "134441" "134443" "134444" "134445" "134446" "134448" "134449"
 [81] "134450" "134452" "134455" "134457" "134460" "134461" "134462" "134463"
 [89] "134465" "134466" "134467" "134468" "134469" "134470" "134471" "134472"
 [97] "134514" "134515" "134516" "134517" "134521" "134522" "134524" "134433"
[105] "134435" "134437" "134439" "134442" "134451" "134453" "134454" "134464"
[113] "134523" "134447" "134456" "134458" "134459" "134518" "134519" "134520"
[121] "137188" "137189" "137190" "137191" "137192" "137193" "137194" "137195"
[129] "137196" "137197" "137198" "136970" "136971" "136972" "136974" "136975"
[137] "136976" "136977" "136978" "136980" "136981" "136982" "136983" "136984"
[145] "136985" "136988" "136989" "136992" "136995" "136996" "136997" "136998"
[153] "136999" "137001" "137002" "137003" "137004" "137005" "137006" "137008"
[161] "137011" "137014" "137015" "137016" "137017" "137019" "137021" "137022"
[169] "137023" "137024" "137025" "137026" "137028" "137029" "137030" "137031"
[177] "137032" "137034" "137035" "137036" "137037" "136973" "136979" "136986"
[185] "136987" "136990" "136991" "136993" "136994" "137009" "137020" "137027"
[193] "137033" "137106" "137107" "137108" "137109" "137110" "137111" "137112"
[201] "137113" "137114" "137115" "137116" "137117" "137118" "137119" "137120"
[209] "137121" "137122" "137123" "137124" "137125" "137126" "137127" "137128"
[217] "137129" "137131" "137132" "137133" "137134" "137135" "137136" "137137"
[225] "137139" "137141" "137142" "137143" "137144" "137145" "137146" "137130"
[233] "137138" "137140" "139636" "139637" "139639" "139642" "139647" "139651"
[241] "139652" "139653" "139655" "139657" "139658" "139659" "139660" "139661"
[249] "139662" "139663" "139664" "139665" "139666" "139667" "139668" "139669"
[257] "139670" "139671" "139675" "139678" "139679" "139680" "139681" "139685"
[265] "139686" "139687" "139688" "139689" "139690" "139692" "139693" "139694"
[273] "139695" "139696" "139640" "139643" "139644" "139646" "139649" "139650"
[281] "139656" "139682" "139691" "139697" "139638" "139645" "139654" "139672"
[289] "139673" "139674" "139676" "139677" "139683" "139684" "139641" "139648"
[297] "139761" "139763" "139764" "139766" "139768" "139770" "139775" "139776"
[305] "139777" "139779" "139782" "139783" "139784" "139786" "139787" "139788"
[313] "139789" "139791" "139793" "139794" "139795" "139796" "139843" "139845"
[321] "139846" "139848" "139849" "139850" "139852" "139762" "139765" "139767"
[329] "139769" "139772" "139774" "139778" "139780" "139781" "139792" "139797"
[337] "139798" "139800" "139842" "139844" "139847" "139851" "139790" "139799"
[345] "139760" "139771" "139773" "139785"

In [27]:
for(i in Pre_trip_id_r4_d0){
  print(i)
  vanilla_rf_trips_cv('4', '0', 'Pre', i)
  print('Done!')
  print('_*_*_')
}

[1] "134296"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134298"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134299"


...

[1] "Done!"
[1] "_*_*_"
[1] "139773"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "139785"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"


In [28]:
Pre_trip_id_r4_d1 = as.character(unique(Pre_Trips_r4_d1$trip_id))
Pre_trip_id_r4_d1

[1] "134364" "134365" "134366" "134368" "134370" "134371" "134372" "134373"
  [9] "134374" "134375" "134376" "134378" "134379" "134380" "134382" "134383"
 [17] "134384" "134385" "134386" "134387" "134388" "134390" "134392" "134393"
 [25] "134394" "134397" "134398" "134399" "134400" "134401" "134403" "134404"
 [33] "134406" "134407" "134408" "134409" "134410" "134415" "134416" "134417"
 [41] "134418" "134420" "134421" "134422" "134425" "134426" "134427" "134428"
 [49] "134369" "134377" "134389" "134391" "134395" "134396" "134405" "134411"
 [57] "134412" "134413" "134414" "134423" "134424" "134429" "134431" "134367"
 [65] "134381" "134402" "134419" "134430" "134473" "134475" "134477" "134479"
 [73] "134482" "134483" "134485" "134486" "134487" "134488" "134490" "134491"
 [81] "134492" "134494" "134495" "134498" "134500" "134503" "134504" "134505"
 [89] "134508" "134509" "134510" "134511" "134512" "134513" "134525" "134526"
 [97] "134528" "134529" "134532" "134533" "134535" "134474" "134476" "134478"
[105] "134480" "134484" "134493" "134496" "134497" "134507" "134534" "134489"
[113] "134499" "134501" "134502" "134506" "134530" "134481" "134527" "134531"
[121] "137199" "137200" "137201" "137202" "137203" "137204" "137205" "137206"
[129] "137207" "137208" "137209" "137038" "137039" "137040" "137041" "137042"
[137] "137043" "137044" "137046" "137047" "137048" "137049" "137050" "137054"
[145] "137059" "137060" "137061" "137062" "137063" "137064" "137066" "137067"
[153] "137068" "137069" "137070" "137071" "137073" "137076" "137077" "137079"
[161] "137080" "137081" "137082" "137085" "137086" "137087" "137088" "137089"
[169] "137090" "137091" "137092" "137094" "137095" "137096" "137097" "137098"
[177] "137099" "137100" "137101" "137104" "137105" "137051" "137052" "137053"
[185] "137055" "137056" "137057" "137058" "137074" "137084" "137093" "137103"
[193] "137045" "137147" "137148" "137149" "137150" "137151" "137152" "137153"
[201] "137154" "137155" "137157" "137158" "137159" "137160" "137161" "137162"
[209] "137163" "137164" "137165" "137166" "137167" "137168" "137169" "137170"
[217] "137171" "137172" "137173" "137174" "137175" "137176" "137177" "137178"
[225] "137180" "137182" "137183" "137184" "137185" "137186" "137187" "137156"
[233] "137179" "137181" "139698" "139704" "139705" "139711" "139713" "139714"
[241] "139715" "139716" "139717" "139718" "139719" "139721" "139722" "139723"
[249] "139724" "139725" "139726" "139727" "139728" "139729" "139730" "139731"
[257] "139732" "139733" "139734" "139735" "139736" "139737" "139743" "139744"
[265] "139745" "139746" "139750" "139751" "139752" "139753" "139754" "139755"
[273] "139757" "139758" "139700" "139703" "139706" "139707" "139709" "139712"
[281] "139720" "139747" "139756" "139759" "139699" "139701" "139708" "139738"
[289] "139739" "139740" "139741" "139742" "139748" "139749" "139702" "139710"
[297] "139802" "139804" "139805" "139807" "139809" "139810" "139812" "139817"
[305] "139818" "139819" "139821" "139825" "139826" "139827" "139829" "139830"
[313] "139831" "139834" "139836" "139837" "139838" "139854" "139857" "139858"
[321] "139859" "139860" "139861" "139863" "139803" "139806" "139808" "139811"
[329] "139814" "139816" "139820" "139822" "139824" "139832" "139835" "139839"
[337] "139840" "139841" "139853" "139855" "139856" "139862" "139823" "139833"
[345] "139801" "139813" "139815" "139828"

In [29]:
for(i in Pre_trip_id_r4_d1){
  print(i)
  vanilla_rf_trips_cv('4', '1', 'Pre', i)
  print('Done!')
  print('_*_*_')
}

[1] "134364"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134365"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134366"


...

[1] "Done!"
[1] "_*_*_"
[1] "139815"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "139828"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"


# Route 9

In [33]:
Pre_trip_id_r9_d0 = as.character(unique(Pre_Trips_r9_d0$trip_id))
Pre_trip_id_r9_d0

[1] "134767" "134768" "134769" "134770" "134771" "134772" "134773" "134774"
  [9] "134777" "134778" "134780" "134781" "134782" "134783" "134786" "134787"
 [17] "134788" "134789" "134793" "134794" "134795" "134779" "134775" "134776"
 [25] "134784" "134785" "134825" "134826" "134827" "134828" "134829" "134830"
 [33] "134831" "134832" "134833" "134834" "134835" "134836" "134837" "134838"
 [41] "134839" "134841" "134842" "134844" "134867" "134868" "134869" "134870"
 [49] "134871" "134872" "134873" "134874" "134875" "134840" "134843" "134845"
 [57] "134790" "134791" "134792" "137551" "137554" "137555" "137557" "137558"
 [65] "137559" "137451" "137452" "137453" "137454" "137455" "137456" "137457"
 [73] "137458" "137459" "137460" "137461" "137462" "137463" "137464" "137465"
 [81] "137466" "137467" "137468" "137469" "137470" "137471" "137472" "137473"
 [89] "137474" "137475" "137476" "137477" "137478" "137479" "137510" "137511"
 [97] "137513" "137514" "137516" "137517" "137518" "137520" "137521" "137523"
[105] "137524" "137525" "137526" "137527" "137528" "137529" "137552" "137553"
[113] "137556" "137512" "137515" "137519" "137522" "140105" "140106" "140103"
[121] "140098" "140099" "140102" "140110" "140111" "140112" "140113" "140117"
[129] "140120" "140121" "140122" "140124" "140125" "140097" "140100" "140101"
[137] "140104" "140107" "140108" "140109" "140116" "140118" "140119" "140123"
[145] "140114" "140115" "140155" "140158" "140160" "140161" "140163" "140164"
[153] "140166" "140167" "140168" "140169" "140170" "140172" "140173" "140174"
[161] "140197" "140198" "140199" "140200" "140201" "140202" "140203" "140204"
[169] "140205" "140162" "140165" "140171" "140156" "140157" "140159"

In [34]:
for(i in Pre_trip_id_r9_d0){
  print(i)
  vanilla_rf_trips_cv('9', '0', 'Pre', i)
  print('Done!')
  print('_*_*_')
}

[1] "134767"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134768"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134769"


...

[1] "Done!"
[1] "_*_*_"
[1] "140157"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "140159"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"


In [37]:
Pre_trip_id_r9_d1 = as.character(unique(Pre_Trips_r9_d1$trip_id))
Pre_trip_id_r9_d1

[1] "134796" "134797" "134798" "134799" "134800" "134801" "134802" "134803"
  [9] "134804" "134807" "134808" "134810" "134811" "134812" "134813" "134816"
 [17] "134817" "134818" "134819" "134823" "134824" "134809" "134805" "134806"
 [25] "134814" "134815" "134846" "134847" "134848" "134849" "134850" "134851"
 [33] "134852" "134853" "134854" "134855" "134856" "134857" "134858" "134859"
 [41] "134860" "134862" "134863" "134865" "134876" "134877" "134878" "134879"
 [49] "134880" "134881" "134882" "134883" "134884" "134861" "134864" "134866"
 [57] "134820" "134821" "134822" "137560" "137563" "137564" "137566" "137567"
 [65] "137568" "137480" "137481" "137482" "137483" "137484" "137485" "137486"
 [73] "137487" "137488" "137489" "137490" "137491" "137492" "137493" "137494"
 [81] "137495" "137496" "137497" "137498" "137499" "137500" "137501" "137502"
 [89] "137503" "137504" "137505" "137506" "137507" "137508" "137531" "137532"
 [97] "137534" "137535" "137536" "137537" "137538" "137539" "137541" "137542"
[105] "137545" "137546" "137547" "137548" "137549" "137550" "137561" "137562"
[113] "137565" "137533" "137540" "137543" "137544" "140135" "140136" "140129"
[121] "140127" "140128" "140132" "140140" "140141" "140142" "140143" "140147"
[129] "140150" "140151" "140152" "140153" "140154" "140126" "140130" "140131"
[137] "140133" "140134" "140137" "140138" "140139" "140146" "140148" "140149"
[145] "140144" "140145" "140176" "140179" "140180" "140181" "140183" "140185"
[153] "140187" "140188" "140189" "140190" "140191" "140193" "140194" "140195"
[161] "140206" "140207" "140208" "140209" "140210" "140211" "140212" "140213"
[169] "140214" "140182" "140186" "140192" "140177" "140178" "140184"

In [38]:
for(i in Pre_trip_id_r9_d1){
  print(i)
  vanilla_rf_trips_cv('9', '1', 'Pre', i)
  print('Done!')
  print('_*_*_')
}

[1] "134796"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134797"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "134798"


...

[1] "Done!"
[1] "_*_*_"
[1] "140178"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
[1] "140184"



── Column specification ────────────────────────────────────────────────────────
cols(
  trip_date = col_date(format = ""),
  trip_id = col_double(),
  hour = col_double(),
  max_occupancy = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double()
)




[1] "Done!"
[1] "_*_*_"
